In [17]:
import polars as pl
import mlflow
import numpy as np
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import catboost as cat
import xgboost as xgb
import lightgbm as lgb


from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.metrics import root_mean_squared_error

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Trail_13_05")

client = mlflow.tracking.MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [3]:
train_dat = pl.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet")
val_dat = pl.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet")

In [4]:
def dat_transform(df):
    df = (
        df.with_columns(
            (
                (pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime"))
                .dt.total_seconds() / 60          # ← total seconds → minutes
            ).alias("duration")
        )
        .filter(pl.col("duration").is_between(1, 60, closed="both"))
    )


    categorical = ['PULocationID', 'DOLocationID']

    df = df.with_columns([pl.col(c).cast(pl.Utf8) for c in categorical])
    return df

In [5]:
train = dat_transform(train_dat)

In [6]:
val = dat_transform(val_dat)

In [7]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target      = "duration"

train = train.with_columns([pl.col(categorical).cast(pl.Utf8)])

train_dicts = train.select(categorical + numerical).to_dicts()   

dv       = DictVectorizer(sparse=True)
X_train  = dv.fit_transform(train_dicts)
y_train  = train[target].to_numpy()

val_dicts = val.select(categorical + numerical).to_dicts()   
X_val  = dv.fit_transform(val_dicts)
y_val  = val[target].to_numpy()

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)



7.6837474557630845

In [13]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():
    mlflow.set_tag("developer","André")
    mlflow.log_param("train_data_path","https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
    mlflow.log_param("val_data_path","https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    
    mlflow.log_metric("RMSE",rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                                                                                                                          | 0/50 [00:00<?, ?trial/s, best loss=?]

/workspaces/MLOps/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:11:30] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.76983                                                                                                                                                                                           
[1]	validation-rmse:6.68255                                                                                                                                                                                           
[2]	validation-rmse:5.93024                                                                                                                                                                                           
[3]	validation-rmse:5.42303                                                                                                                                                                                           
[4]	validation-rmse:5.09753                                                                                                                 

/workspaces/MLOps/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:12:33] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.51610                                                                                                                                                                                           
[1]	validation-rmse:7.84393                                                                                                                                                                                           
[2]	validation-rmse:7.27640                                                                                                                                                                                           
[3]	validation-rmse:6.78804                                                                                                                                                                                           
[4]	validation-rmse:6.38368                                                                                                                 

...

/workspaces/MLOps/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:22:50] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.77145                                                                                                                                                                                           
[1]	validation-rmse:5.50273                                                                                                                                                                                           
[2]	validation-rmse:4.93184                                                                                                                                                                                           
[3]	validation-rmse:4.68979                                                                                                                                                                                           
[4]	validation-rmse:4.59404                                                                                                                 

/workspaces/MLOps/.venv/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:24:03] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.27365                                                                                                                                                                                           
[1]	validation-rmse:7.44454                                                                                                                                                                                           
[2]	validation-rmse:6.77752                                                                                                                                                                                           
[3]	validation-rmse:6.24832                                                                                                                                                                                           
[4]	validation-rmse:5.83210                                                                                                                 

KeyboardInterrupt: 